In [4]:
import pandas as pd
import numpy as np
import os
import json
from a import MaxEntScorer
from metrics import compute_metrics, get_confusion_matrix

# --- CẤU HÌNH ---
DATA_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\data"
OUTPUT_DIR = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

scorer = MaxEntScorer()

def softmax(x):
    # Tránh tràn số (numerical stability)
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Hàm áp dụng ngưỡng thủ công
def apply_threshold_prediction(probs, thresholds):
    """
    probs: Ma trận xác suất (N_samples, N_classes)
    thresholds: List ngưỡng cho từng class [th_class0, th_class1, th_class2]
    """
    # 1. Lấy lớp có xác suất cao nhất ban đầu
    initial_preds = np.argmax(probs, axis=1)
    
    # 2. Lấy giá trị xác suất tương ứng của lớp đó
    # np.arange(len(probs)) tạo index hàng, initial_preds là index cột
    max_confidences = probs[np.arange(len(probs)), initial_preds]
    
    # 3. Lấy ngưỡng tương ứng với lớp được dự đoán
    # Ví dụ: nếu máy đoán lớp 1, nó sẽ lấy thresholds[1]
    selected_thresholds = np.array([thresholds[p] for p in initial_preds])
    
    # 4. So sánh: Nếu xác suất < ngưỡng -> Gán về class 0 (Non-site)
    # Ngược lại giữ nguyên dự đoán ban đầu
    final_preds = np.where(max_confidences >= selected_thresholds, initial_preds, 0)
    
    return final_preds

if __name__ == "__main__":
    RATIOS = ["1-1-1", "2-1-1", "4-1-1", "10-1-1", "100-1-1"]
    
    # CHIẾN THUẬT BIAS
    CLASS_BIAS = np.array([4.5, 1.2, 4.5])

    # --- CẤU HÌNH NGƯỠNG (BẠN CHỈNH Ở ĐÂY) ---
    # Index 0: Non-site (thường để 0.0 vì nó là fallback)
    # Index 1: Donor (GT)
    # Index 2: Acceptor (AG)
    # Ví dụ: [0.0, 0.7, 0.6] nghĩa là:
    # - Nếu đoán là Donor, xác suất phải > 70% mới chốt, không thì về Non-site
    # - Nếu đoán là Acceptor, xác suất phải > 60% mới chốt.
    CONFIDENCE_THRESHOLDS = [0.0, 0.7, 0.6] 

    for ratio in RATIOS:
        input_file = os.path.join(DATA_DIR, f"maxent_input_{ratio}.csv")
        if not os.path.exists(input_file):
            print(f"❌ Không tìm thấy file: {input_file}")
            continue
            
        print(f"\n🚀 Ratio: {ratio} | Bias: {CLASS_BIAS} | Thresholds: {CONFIDENCE_THRESHOLDS}")
        df = pd.read_csv(input_file)
        
        # 1. Tính Scores thô
        s5_scores = df['seq_5'].apply(scorer.score5).values
        s3_scores = df['seq_3'].apply(scorer.score3).values
        
        # 2. Tạo ma trận điểm
        combined_scores = np.stack([np.zeros(len(s5_scores)), s5_scores, s3_scores], axis=1)
        
        # 3. Áp dụng Bias
        weighted_logits = combined_scores + CLASS_BIAS
        
        # 4. Dự đoán (CÓ SỬ DỤNG NGƯỠNG)
        all_probs = softmax(weighted_logits)
        
        # --- THAY ĐỔI Ở ĐÂY: Dùng hàm apply_threshold_prediction thay vì np.argmax ---
        # all_preds = np.argmax(weighted_logits, axis=1) # (Code cũ)
        all_preds = apply_threshold_prediction(all_probs, CONFIDENCE_THRESHOLDS)
        
        # 5. Tính Metrics
        y_true = df['label'].values
        metrics_res = compute_metrics(y_true, all_preds, probs=all_probs, k=2)
        metrics_res['confusion_matrix'] = get_confusion_matrix(y_true, all_preds).tolist()
        
        # Lưu thêm thông tin thresholds vào kết quả để tiện theo dõi
        metrics_res['meta'] = {
            "bias_applied": CLASS_BIAS.tolist(), 
            "thresholds": CONFIDENCE_THRESHOLDS,
            "ratio": ratio
        }

        output_json = os.path.join(OUTPUT_DIR, f"results_{ratio}.json")
        with open(output_json, "w") as f:
            json.dump(metrics_res, f, indent=4)
            
        print(f"✅ Kết quả lưu tại: {output_json}")


🚀 Ratio: 1-1-1 | Bias: [4.5 1.2 4.5] | Thresholds: [0.0, 0.7, 0.6]
✅ Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_1-1-1.json

🚀 Ratio: 2-1-1 | Bias: [4.5 1.2 4.5] | Thresholds: [0.0, 0.7, 0.6]
✅ Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_2-1-1.json

🚀 Ratio: 4-1-1 | Bias: [4.5 1.2 4.5] | Thresholds: [0.0, 0.7, 0.6]
✅ Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_4-1-1.json

🚀 Ratio: 10-1-1 | Bias: [4.5 1.2 4.5] | Thresholds: [0.0, 0.7, 0.6]
✅ Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_10-1-1.json

🚀 Ratio: 100-1-1 | Bias: [4.5 1.2 4.5] | Thresholds: [0.0, 0.7, 0.6]
✅ Kết quả lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\MaxEntScan\results\results_100-1-1.json
